# Load Packages

In [ ]:
# Add any needed packages

from google.cloud import bigquery
import seaborn as sns
from scipy.stats import ttest_ind_from_stats

# Setup Connection

In [ ]:
# Fill in identifiers

CURATION_PROJECT_ID = ""
EHR_OPS_DATASET_ID = ""
PDR_PROJECT_ID = ""
RDR_DATASET_ID = ""
EHR_OPS_PROJECT_ID = ""
EHR_DENSITY_DATASET_ID = ""

# Add any other needed identifiers

In [ ]:
client = bigquery.Client(project=CURATION_PROJECT_ID)

# EHR Density vs Diversity

## EHR Density Equation:
2/n + (n-2)/n * (1 - ((n-1)*Var(gi; i= 1,2,..., n-1))**1/2
gi = (x_(i+1)-x_i)/(x_n-x_1)
n is the number of times that a participant was observed or had a clinical event (condition, laboratory test, medication, or procedure)
i is the observation or clinical event number
x_(i+1)-x_i is the time gap between two consecutive observations
x_n-x_1 is the time gap between the first and last observation

equation reference: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9196700/bin/ocac046_supplementary_data.zip




In [ ]:
person_event_query = f"""
SELECT ehr.person_id,
    ehr.density, 
    pdr.ubr_sex,
    pdr.ubr_sexual_orientation,
    pdr.ubr_gender_identity,
    pdr.ubr_ethnicity,
    pdr.ubr_geography,
    pdr.ubr_education,
    pdr.ubr_income,
    pdr.ubr_sexual_gender_minority,
    pdr.ubr_age_at_consent,
    pdr.ubr_disability,
    pdr.ubr_overall,
    bbo.bbo_collection_method,
    pss.patient_status
FROM `{EHR_OPS_PROJECT_ID}.{EHR_DENSITY_DATASET_ID}.ehr_density` ehr
JOIN `{PDR_PROJECT_ID}.{RDR_DATASET_ID}.v_pdr_participant` pdr
    ON ehr.person_id = pdr.participant_id
JOIN `{PDR_PROJECT_ID}.{RDR_DATASET_ID}.v_pdr_participant_biobank_order` bbo
    ON ehr.person_id = bbo.participant_id
JOIN 
    (SELECT
        DISTINCT participant_id,
        patient_status,
        RANK() OVER (PARTITION BY ps.participant_id ORDER BY patient_status_modified DESC) ps_order
    FROM `{PDR_PROJECT_ID}.{RDR_DATASET_ID}.v_pdr_participant_patient_status` ps) pss
    ON ehr.person_id = pss.participant_id AND ps_order = 1
"""

In [ ]:
person_ehr_density_df = client.query(person_event_query).to_dataframe()

In [ ]:
person_ehr_density_df

In [ ]:
person_ehr_density_df.dropna(inplace=True)

In [ ]:
mapping_dict = {0: 'RBR', 1: 'UBR', 2: 'SKIP/UNKNOWN'}
for i in person_ehr_density_df.columns.values:
    if 'ubr' in i:
        person_ehr_density_df[i] = person_ehr_density_df[i].astype(object).replace(mapping_dict)

In [ ]:
person_ehr_density_df

In [ ]:
def run_analysis(target_column, df):
    summary = df[[target_column, 'density']].groupby(target_column).describe().reset_index()
    
    n1 = summary['density']['count'][0]
    n2 = summary['density']['count'][1]
    
    mean1 = summary['density']['mean'][0]
    mean2 = summary['density']['mean'][1]
    
    std1 = summary['density']['std'][0]
    std2 = summary['density']['std'][1]
    
    tstat, pvalue = ttest_ind_from_stats(mean1, std1, n1, mean2, std2, n2)
    print(f"t-statistic: {tstat}")
    print(f"p-value: {pvalue}")
    
    alpha = 0.05
    if pvalue < alpha:
        print("The difference between the groups is statistically significant.")
    else:
        print("There is no significant difference between the groups.")
    
    sns.boxplot(data=df, x=target_column, y="density")
    return summary, tstat, pvalue


## UBR vs. RBR 
### (for each individual UBR group as well as specifics on UBR R/E)

#### UBR Overall

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_overall', person_ehr_density_df)

In [ ]:
summary

#### UBR SEX

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_sex', person_ehr_density_df)

In [ ]:
summary

#### UBR SEXUAL ORIENTATION

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_sexual_orientation', person_ehr_density_df)

In [ ]:
summary

#### UBR SEXUAL GENDER MINORITY

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_sexual_gender_minority', person_ehr_density_df)

In [ ]:
summary

#### UBR GENDER IDENTITY

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_gender_identity', person_ehr_density_df)

In [ ]:
summary

#### UBR ETHNICITY

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_ethnicity', person_ehr_density_df)

In [ ]:
summary

#### UBR GEOGRAPHY

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_geography', person_ehr_density_df)

In [ ]:
summary

#### UBR EDUCATION

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_education', person_ehr_density_df)

In [ ]:
summary

#### UBR INCOME

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_income', person_ehr_density_df)

In [ ]:
summary

#### UBR AGE AT CONSENT

In [ ]:
summary, tstat, pvalue = run_analysis('ubr_age_at_consent', person_ehr_density_df)

In [ ]:
summary

## Remote vs. in-person biosample donation

In [ ]:
summary, tstat, pvalue = run_analysis('bbo_collection_method', person_ehr_density_df)

In [ ]:
summary

## Patient vs. non-patient status at HPO

In [ ]:
summary, tstat, pvalue = run_analysis('patient_status', person_ehr_density_df)

In [ ]:
sns.boxplot(data=person_ehr_density_df, x="patient_status", y="density")

### EHR Density by Total Number of Records

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ehr_density_count_query = f"""
SELECT total_events, density 
 FROM `{EHR_OPS_PROJECT_ID}.{EHR_DENSITY_DATASET_ID}.ehr_density`
"""

In [ ]:
ehr_density_count_df = client.query(ehr_density_count_query).to_dataframe()

In [ ]:
ehr_density_count_df

In [ ]:
ehr_density_mean = ehr_density_count_df.groupby('total_events')['density'].describe().reset_index()

In [ ]:
ehr_density_mean

In [ ]:
import plotly.express as px

fig = px.line(ehr_density_mean, x="total_events", y="count", log_x=True)
fig.update_yaxes(title_text="Person Count")
fig.update_xaxes(title_text="Number of Events per Person")
fig.show()

In [ ]:
sns.set_theme(style="white")
plt.xscale("log")
sns.scatterplot(data=ehr_density_count_df, x="total_events", y="density", s=1, edgecolor='None')

In [ ]:
ehr_density_count_df

In [ ]:
import plotly.express as px
fig = px.scatter(ehr_density_count_df, x="total_events", y="density", log_x=True)
fig.show()

## The average number of EHR records per participant across each domain
#### - Conditions
#### - Procedures
#### - Drugs
#### - Labs and measurements
#### - Visits
#### - Observations
#### - Devices

In [ ]:
event_per_person_query = f"""
SELECT DISTINCT 
    person_id, 
    event_type, 
    count(distinct event_id) AS event_cnt_per_person
FROM `{EHR_OPS_PROJECT_ID}.{EHR_DENSITY_DATASET_ID}.unioned_event`
GROUP BY 1,2
"""

In [ ]:
event_per_person_df = client.query(event_per_person_query).to_dataframe()

In [ ]:
event_per_person_df[['event_type', 'event_cnt_per_person']].groupby('event_type').mean()

## The average duration between the first and last EHR record per participant across each of the above domains

In [ ]:
ehr_duration_per_person_query = f"""
SELECT 
  person_id, 
  event_type,
  MAX(event_datetime) as max_event_datetime, 
  MIN(event_datetime) as min_event_datetime,
  TIMESTAMP_DIFF(MAX(event_datetime), MIN(event_datetime), DAY) AS total_days_diff
FROM `{EHR_OPS_PROJECT_ID}.{EHR_DENSITY_DATASET_ID}.unioned_event`
GROUP BY 1,2
"""

In [ ]:
ehr_duration_per_person_df = client.query(ehr_duration_per_person_query).to_dataframe()

In [ ]:
ehr_duration_per_person_df.groupby('event_type')['total_days_diff'].mean()